# Convert all the meta files inside the meta folder to one csv file

In [2]:
import pandas as pd
import numpy as np
import os
import json

In [111]:
import pandas as pd
import numpy as np
import os
import json
from urllib import parse, request


class CsvToMeta:
    
    """
    Initiate the meta folder from the CSV file containing the dataset.
    """
    
    def __init__(self, input_csv_file_path, output_folder, download_images=False, google_api_key_filepath=None):
        """
        `input_csv_file_path`: string, path to the csv file containing the dataset informations\n
        `output_folder`: string, path to the folder where to store the dataset. 2 subfolders are created: `Helipad_Dataset_meta` and `Helipad_Dataset`\n
        `download_images`: boolean, True to download and store the images from Google Maps\n
        `google_api_key_filepath`: string, path to the local file containing the user's Google Maps Api key.
        """
        self.input_csv_file_path = input_csv_file_path
        self.output_folder = output_folder
        self.download_images = download_images
        
        if not os.path.isdir(os.path.join(self.output_folder, 'Helipad_Dataset_meta')):
            os.mkdir(os.path.join(self.output_folder, 'Helipad_Dataset_meta'))
        if download_images:
            if not os.path.isdir(os.path.join(self.output_folder, 'Helipad_Dataset')):
                os.mkdir(os.path.join(self.output_folder, 'Helipad_Dataset'))
            with open(google_api_key_filepath, 'r') as f:
                self.api_key = f.read()
            f.close()
        
        self.image_folder = os.path.join(self.output_folder, 'Helipad_Dataset')
        self.meta_folder = os.path.join(self.output_folder, 'Helipad_Dataset_meta')
        
    def run(self):
        
        df = pd.read_csv(self.input_csv_file_path)
        
        helipad_numbers = df.Helipad_number.value_counts().keys().sort_values().tolist()
        
        for helipad_number in helipad_numbers:
            
            index = np.where(df['Helipad_number'].values == helipad_number)[0]
            df_helipad = df.iloc[index, :]
            
            nb_helipad = df_helipad.shape[0]
            
            if nb_helipad > 0:
                meta = dict()
                meta['coordinates'] = {}
                meta['coordinates']['latitude'] = float(df_helipad.iloc[0, :]['latitude'])
                meta['coordinates']['longitude'] = float(df_helipad.iloc[0, :]['longitude'])
                meta['coordinates']['zoom'] = int(df_helipad.iloc[0, :]['zoom'])
                meta['info'] = {}
                meta['info']['FAA_index'] = int(df_helipad.iloc[0, :]['FAA_index'])
                meta['info']['Helipad_number'] = int(df_helipad.iloc[0, :]['Helipad_number'])
                meta['info']['url'] = str(df_helipad.iloc[0, :]['url'])
                meta['groundtruth'] = {}
                meta['groundtruth']['helipad'] = bool(df_helipad.iloc[0, :]['groundtruth'])
                meta['groundtruth']['category'] = str(df_helipad.iloc[0, :]['category'])
                bboxes = []
                for i in range(nb_helipad):
                    box = [float(df_helipad.iloc[i, :]['minX']),\
                           float(df_helipad.iloc[i, :]['minY']),\
                           float(df_helipad.iloc[i, :]['maxX']),\
                           float(df_helipad.iloc[i, :]['maxY'])]
                    bboxes.append(box)
                meta['groundtruth']['box'] = bboxes
                
                folder_id = helipad_number // 100
                
                folder_name = 'Folder_{:03d}'.format(folder_id)
                meta_filename = "Helipad_{:05d}.meta".format(helipad_number)
                meta_path = os.path.join(self.meta_folder, folder_name, meta_filename)
                
                if not os.path.isdir(os.path.dirname(meta_path)):
                    os.mkdir(os.path.dirname(meta_path))
                
                with open(meta_path, 'w') as f:
                    json.dump(meta, f, indent=4, sort_keys=True)
                
                print('Saved :' + meta_path)
                
                if self.download_images:
                    image_filename = "Helipad_{:05d}.png".format(helipad_number)
                    image_path = os.path.join(self.image_folder, folder_name, image_filename)
                    if not os.path.isdir(os.path.dirname(image_path)):
                        os.mkdir(os.path.dirname(image_path))
                    url = df_helipad.iloc[0, :]['url']
                    url = url.replace('APIKEY', self.api_key)
                    
                    request.urlretrieve(url, image_path)
                    print('Downloaded: ' + image_path)
        
        print('Dataset created!')
    

if __name__ == "__main__":
    input_csv_file_path = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Helipad_DataBase_meta\\Helipad_DataBase_annotated.csv"
    output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\New_Dataset_Folder"
    download_images = False
    google_api_key_filepath = None

    csv_to_meta = CsvToMeta(input_csv_file_path=input_csv_file_path,
                           output_folder=output_folder,
                           download_images=download_images,
                           google_api_key_filepath=google_api_key_filepath)

    csv_to_meta.run()

In [112]:
input_csv_file_path = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Helipad_DataBase_meta\\Helipad_DataBase_annotated.csv"
output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\New_Dataset_Folder"
download_images = False
google_api_key_filepath = None

csv_to_meta = CsvToMeta(input_csv_file_path=input_csv_file_path,
                       output_folder=output_folder,
                       download_images=download_images,
                       google_api_key_filepath=google_api_key_filepath)

csv_to_meta.run()


Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00000.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00001.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00002.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00003.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00004.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00005.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00006.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00007.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00008.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_000\Helipad_00099.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00100.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00101.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00102.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00103.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00104.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00105.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00106.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00107.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00180.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00181.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00182.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00183.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00184.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00185.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00186.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00187.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_001\Helipad_00188.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00257.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00258.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00259.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00260.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00261.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00262.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00263.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00264.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_002\Helipad_00265.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00336.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00337.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00338.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00339.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00340.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00341.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00342.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00343.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_003\Helipad_00344.meta
Saved :C:\Users\AISG\Documents\Jonas\

...

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05561.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05562.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05563.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05564.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05565.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05566.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05567.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05568.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_055\Helipad_05569.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05641.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05642.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05643.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05644.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05645.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05646.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05647.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05648.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_056\Helipad_05649.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05721.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05722.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05723.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05724.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05725.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05726.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05727.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05728.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05729.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05798.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_057\Helipad_05799.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05800.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05801.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05802.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05803.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05804.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05805.meta
Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05806.meta
Saved :C:\Users\AISG\Documents\Jonas\

Saved :C:\Users\AISG\Documents\Jonas\New_Dataset_Folder\Helipad_Dataset_meta\Folder_058\Helipad_05877.meta
Dataset created!


In [60]:
"Helipad_{:05d}.meta".format(3)

'Helipad_00003.meta'

In [58]:
'Folder_{:03d}'.format(3)

'Folder_003'

In [97]:
meta_folder_original = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Helipad_DataBase_meta\\Helipad_DataBase_meta_original"
output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Helipad_DataBase_meta"
output_filename = "Helipad_DataBase_annotated.csv"

In [96]:
def convert_meta_to_csv(meta_folder, output_folder, output_filename):
    data = []
    for subdir, dirs, files in os.walk(meta_folder, topdown=True):
        for file in files:
            if file[0] == ".":
                continue
            if os.path.splitext(file)[1] != ".meta":
                continue
            filepath = os.path.join(subdir, file)
            with open(filepath, 'r') as f:
                meta = json.load(f)
            f.close()
            info = meta['info']
            coordinates = meta['coordinates']
            if 'groundtruth' not in meta:
                continue
            groundtruth = meta['groundtruth']    
            if not groundtruth['helipad']:
                helipad = [info['Helipad_number'], info['FAA_index'], coordinates['latitude'], coordinates['longitude'],\
                           coordinates['zoom'], groundtruth['helipad'], str(-1), -1, -1, -1, -1, info['url']]
                data.append(helipad)
            else:
                if 'box' not in groundtruth or len(groundtruth['box']) == 0:
                    if 'category' not in groundtruth:
                        helipad = [info['Helipad_number'], info['FAA_index'], coordinates['latitude'], coordinates['longitude'],\
                           coordinates['zoom'], groundtruth['helipad'], str(-1), -1, -1, -1, -1, info['url']]
                    else:
                        cat = groundtruth['category']
                        if cat == 'd':
                            cat = "other"
                        elif cat == 'u':
                            cat = "unknown"
                        helipad = [info['Helipad_number'], info['FAA_index'], coordinates['latitude'], coordinates['longitude'],\
                           coordinates['zoom'], groundtruth['helipad'], cat, -1, -1, -1, -1, info['url']]
                    data.append(helipad)
                else:
                    cat = groundtruth['category']
                    if cat == 'd':
                        cat = "other"
                    elif cat == 'u':
                        cat = "unknown"
                    for i in range(len(groundtruth['box'])):
                        box = groundtruth['box'][i]
                        helipad = [info['Helipad_number'], info['FAA_index'], coordinates['latitude'], coordinates['longitude'],\
                           coordinates['zoom'], groundtruth['helipad'], cat, \
                                   min(int(box[0]), int(box[2])),\
                                   min(int(box[1]), int(box[3])),\
                                   max(int(box[0]), int(box[2])),\
                                   max(int(box[1]), int(box[3])), info['url']]
                        data.append(helipad)
            
    columns = ['Helipad_number', 'FAA_index', 'latitude', 'longitude', 'zoom', 'groundtruth', 'category', 'minX', 'minY', 'maxX', 'maxY', 'url']
    
    df = pd.DataFrame(data=data, columns=columns)
    
    df.to_csv(os.path.join(output_folder, output_filename))
    
    return df
                    
            

In [98]:
df = convert_meta_to_csv(meta_folder=meta_folder_original,
                   output_folder=output_folder,
                   output_filename=output_filename)

In [99]:
df.head(20)

,Helipad_number,FAA_index,latitude,longitude,zoom,groundtruth,category,minX,minY,maxX,maxY,url
0,0,19,61.212703,-149.826583,20,True,0,35,50,578,594,http://maps.googleapis.com/maps/api/staticmap?...
1,1,20,61.218906,-149.886481,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...
2,2,23,61.156131,-149.792300,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...
3,3,26,61.235383,-149.749275,20,True,1,89,94,589,588,http://maps.googleapis.com/maps/api/staticmap?...
4,4,31,61.129694,-149.795917,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...
5,5,32,61.189031,-149.821831,20,True,2,40,1,625,419,http://maps.googleapis.com/maps/api/staticmap?...
6,6,36,61.224775,-149.839094,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...
7,7,68,61.516581,-149.931081,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...
8,8,82,61.521575,-149.927139,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...
9,9,83,61.514861,-149.929011,20,False,-1,-1,-1,-1,-1,http://maps.googleapis.com/maps/api/staticmap?...


In [29]:
df.url[0]

'http://maps.googleapis.com/maps/api/staticmap?center=61.21270277777778,-149.82658333333333&zoom=20&size=640x640&sensor=False&maptype=satellite&key=APIKEY'

In [8]:
df.shape

(6053, 11)

In [89]:
df.Helipad_number.value_counts().keys().sort_values().tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 29,
 30,
 31,
 32,
 33,
 37,
 40,
 41,
 42,
 43,
 45,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201

In [87]:
df.Helipad_number.value_counts()[540]

6

In [86]:
np.where(df.Helipad_number.values == 540)[0]

array([539, 540, 541, 542, 543, 544], dtype=int64)

In [62]:
df.iloc[np.where(df.Helipad_number.values == 540)[0], :].iloc[0, :].url

'http://maps.googleapis.com/maps/api/staticmap?center=34.05438888888889,-118.22952777777779&zoom=20&size=640x640&sensor=False&maptype=satellite&key=APIKEY'

In [65]:
url = df.iloc[np.where(df.Helipad_number.values == 540)[0], :].iloc[0, :].url
url = url.replace('APIKEY', 'myAPIKEY')
url

'http://maps.googleapis.com/maps/api/staticmap?center=34.05438888888889,-118.22952777777779&zoom=20&size=640x640&sensor=False&maptype=satellite&key=myAPIKEY'

In [27]:
len(df.Helipad_number.values)

6053

In [17]:
df.Helipad_number.value_counts()

540     6
5064    5
2340    5
5015    5
947     5
       ..
5372    1
1278    1
3327    1
5376    1
0       1
Name: Helipad_number, Length: 5849, dtype: int64

In [7]:
df.iloc[np.where(df['Helipad_number']==66)[0]]

,Helipad_number,FAA_index,latitude,longitudes,groundtruth,category,minX,minY,maxX,maxY,url
46,66,792,33.535783,-86.811406,True,2,218,142,357,289,http://maps.googleapis.com/maps/api/staticmap?...
47,66,792,33.535783,-86.811406,True,2,279,347,425,504,http://maps.googleapis.com/maps/api/staticmap?...
